In [20]:
import requests
import pandas as pd
import csv

import xml.dom.minidom


# pylint: disable=invalid-name
def getUniqueElementValueFromXmlString(xmlString, elementName):
    """
    Extracts an element value from an XML string.
    For example, invoking
    getUniqueElementValueFromXmlString(
        '<?xml version="1.0" encoding="UTF-8"?><foo>bar</foo>', 'foo')
    should return the value 'bar'.
    """
    xmlStringAsDom = xml.dom.minidom.parseString(xmlString)
    elementsByName = xmlStringAsDom.getElementsByTagName(elementName)
    elementValue = None
    if len(elementsByName) > 0:
        elementValue = elementsByName[0].toxml().replace(
            '<' + elementName + '>', '').replace('</' + elementName + '>', '')
    return elementValue

username = "maksimp@sandiego.gov"
password = "KrakatoaIsTheBestCoffeeShopEver2"
security_token = "qn4CD0BK85EN8ZCXig2npOmN"
report_id = "00Ot0000000mCVoEAM"
client_id = 'DataSD'
sf_version = '29.0'
domain = 'sdgov.my'

def sf_login():
    # Security Token Soap request body
    login_soap_request_body = """<?xml version="1.0" encoding="utf-8" ?>
    <env:Envelope
            xmlns:xsd="http://www.w3.org/2001/XMLSchema"
            xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
            xmlns:env="http://schemas.xmlsoap.org/soap/envelope/"
            xmlns:urn="urn:partner.soap.sforce.com">
        <env:Header>
            <urn:CallOptions>
                <urn:client>{client_id}</urn:client>
                <urn:defaultNamespace>sf</urn:defaultNamespace>
            </urn:CallOptions>
        </env:Header>
        <env:Body>
            <n1:login xmlns:n1="urn:partner.soap.sforce.com">
                <n1:username>{username}</n1:username>
                <n1:password>{password}{token}</n1:password>
            </n1:login>
        </env:Body>
    </env:Envelope>""".format(
        username=username, password=password, token=security_token,
        client_id=client_id)

    soap_url = 'https://{domain}.salesforce.com/services/Soap/u/{sf_version}'

    soap_url = soap_url.format(domain=domain, sf_version=sf_version)

    login_soap_request_headers = {
        'content-type': 'text/xml',
        'charset': 'UTF-8',
        'SOAPAction': 'login'
    }
    response = requests.post(
            soap_url, login_soap_request_body, headers=login_soap_request_headers)

    if response.status_code != 200:
        raise Exception("Login Failed")
    
    session_id = getUniqueElementValueFromXmlString(
        response.content, 'sessionId')

    return session_id


In [21]:

sf = sf_login()

print "get sid ", sf




get sid  00D61000000YoZw!AR8AQCUqar59Cq6gurNg6Neye6kO7wi.viZkUDe1_P8jsiJglGXl3pX_RvvoNVouHPyhXrPOyVaiF4w0RqwGXJ_lnfoOuCbj


In [ ]:
d = requests.get(
    "https://sdgov.my.salesforce.com/{}?view=d&snip&export=1&enc=UTF-8&xf=csv".format(report_id),
                  headers = sf.headers, cookies = {'sid' : sf})

lines = d.content.splitlines()
reader = csv.reader(lines)
data = list(reader)
data = data[:-7]
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.drop(0)

df.shape